# Download Data

In [1]:
!wget -q https://hkustconnect-my.sharepoint.com/:u:/g/personal/nnanda_connect_ust_hk/EfREjZqiZTlPqhqUPICBbPABdlgPumlaUVxPncm-_9aWIw?download=1 -O "Project 1 - data.zip"
!unzip -q "Project 1 - data.zip"

# Import Libraries

In [2]:
!pip -q install keras-layer-normalization

In [3]:
import os
import nltk
import math
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
from collections import Counter
from itertools import chain
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, BatchNormalization,\
    Activation, Input, Add, Concatenate, Embedding, Conv1D, MaxPool1D,\
    Flatten, LSTM, Bidirectional, MaxPooling1D, SimpleRNN, GRU, SpatialDropout1D
from keras_layer_normalization import LayerNormalization
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [4]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
stopwords = set(stopwords.words("english"))
ps = PorterStemmer()

# Data Loader

In [6]:
def load_data(split_name='train', columns=['text', 'stars']):
    try:
        print(f"select [{', '.join(columns)}] columns from the {split_name} split")
        df = pd.read_csv(f'data_2021_spring/{split_name}.csv')
        df = df.loc[:,columns]
        print("succeed!")
        return df
    except:
        print("Failed, then try to ")
        print(f"select all columns from the {split_name} split")
        df = pd.read_csv(f'data_2021_spring/{split_name}.csv')
        return df

In [7]:
train_df = load_data('train', columns=['full'])
valid_df = load_data('valid', columns=['full'])
test_df = load_data('test', columns=['full'])

select [full] columns from the train split
Failed, then try to 
select all columns from the train split
select [full] columns from the valid split
Failed, then try to 
select all columns from the valid split
select [full] columns from the test split
Failed, then try to 
select all columns from the test split


In [8]:
train_df.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,39rLHYJOy2774ZIUouuWLw,0,2017-06-28 21:44:02,0,ynzOFepQYSCDGdfWDWxiZw,4,Nice to have a diner still around. Food was go...,0,Sl6VgFOB-XXfFIAYp7TFkw
1,E-Kq1Yu1d6N3TL2qX0aqjA,0,2018-04-09 03:45:19,0,sQX9ncJBEdBf16AWsvO6Vg,2,"Tried this a while back, got the fried chicken...",0,gcx01pMqWzkni2UC-zoZrA
2,nWW6fBfBljiRFa4sG7TyxA,0,2014-07-19 02:00:04,0,bVIf2kqbzvif3miNe3ARNw,4,I expected more pork selections on menu. Food ...,0,Mn9VzPbrCYU4EcP_C1oBOg
3,qmIHO-6T_KEfPC9jyGDamQ,0,2011-11-11 08:10:24,0,LNj1OFxy2ool3PZANGchPA,4,YUMMY!!! This place is phenomenal. It is Price...,0,SKV1heo00fdciCbCN9Z33A
4,pKk7jCFIm96qDdk0laVT2w,1,2010-01-16 20:04:00,1,bZXxa0hO6wQlHD-MkMf4iw,5,The Truffle Macaroni & Cheese and Potatoes Au ...,1,p1r7rZYruZR92x1A649PTQ


# Logistic Regression on just three attributes - cool, funny, useful

## Just funny

In [26]:
x_train = train_df['funny'].values
y_train = train_df['stars'].values

In [27]:
x_train.shape, y_train.shape

((10000,), (10000,))

In [28]:
x_train = x_train.reshape(-1, 1)

In [29]:
x_train.shape, y_train.shape

((10000, 1), (10000,))

In [37]:
lr = LogisticRegression(C=0.001)

In [38]:
lr.fit(x_train, y_train)

LogisticRegression(C=0.001, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [39]:
x_valid = valid_df['funny'].values.reshape(-1, 1)
y_valid = valid_df['stars'].values
y_pred = lr.predict(x_valid)
print(classification_report(y_valid, y_pred))
print("\n\n")
print(confusion_matrix(y_valid, y_pred))
print('accuracy', np.mean(y_valid == y_pred))

              precision    recall  f1-score   support

           1       0.26      1.00      0.41       517
           2       0.00      0.00      0.00       278
           3       0.00      0.00      0.00       344
           4       0.00      0.00      0.00       427
           5       0.00      0.00      0.00       434

    accuracy                           0.26      2000
   macro avg       0.05      0.20      0.08      2000
weighted avg       0.07      0.26      0.11      2000




[[517   0   0   0   0]
 [278   0   0   0   0]
 [344   0   0   0   0]
 [427   0   0   0   0]
 [434   0   0   0   0]]
accuracy 0.2585


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [34]:
# Grid-search for c
params = dict(C = np.logspace(-3, 1, 4),
              penalty = ['l1', 'l2'])

gscv = GridSearchCV(LogisticRegression(max_iter=1000,
                                       random_state=647),
                    params,
                    scoring='f1_macro',
                    cv=3,
                    verbose=1,
                    n_jobs=-1)

In [35]:
gscv.fit(x_train, y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:    1.2s finished


GridSearchCV(cv=3, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=1000, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=647, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': array([1.00000000e-03, 2.15443469e-02, 4.64158883e-01, 1.00000000e+01]),
                         'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1_macro', verbose=1)

In [36]:
print(gscv.best_params_)

{'C': 0.001, 'penalty': 'l2'}


## Just cool

In [9]:
x_train = train_df['cool'].values
y_train = train_df['stars'].values

In [10]:
x_train.shape, y_train.shape

((10000,), (10000,))

In [11]:
x_train = x_train.reshape(-1, 1)

In [12]:
x_train.shape, y_train.shape

((10000, 1), (10000,))

In [21]:
lr = LogisticRegression(C=0.021544346900318832)

In [22]:
lr.fit(x_train, y_train)

LogisticRegression(C=0.021544346900318832, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [23]:
x_valid = valid_df['cool'].values.reshape(-1, 1)
y_valid = valid_df['stars'].values
y_pred = lr.predict(x_valid)
print(classification_report(y_valid, y_pred))
print("\n\n")
print(confusion_matrix(y_valid, y_pred))
print('accuracy', np.mean(y_valid == y_pred))

              precision    recall  f1-score   support

           1       0.29      0.87      0.44       517
           2       0.00      0.00      0.00       278
           3       0.00      0.00      0.00       344
           4       0.29      0.31      0.30       427
           5       0.00      0.00      0.00       434

    accuracy                           0.29      2000
   macro avg       0.12      0.24      0.15      2000
weighted avg       0.14      0.29      0.18      2000




[[451   0   0  66   0]
 [229   0   0  49   0]
 [247   0   0  97   0]
 [294   0   0 133   0]
 [326   0   0 108   0]]
accuracy 0.292


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [24]:
# Grid-search for c
params = dict(C = np.logspace(-3, 1, 4),
              penalty = ['l1', 'l2'])

gscv = GridSearchCV(LogisticRegression(max_iter=1000,
                                       random_state=647),
                    params,
                    scoring='f1_macro',
                    cv=3,
                    verbose=1,
                    n_jobs=-1)

In [19]:
gscv.fit(x_train, y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:    1.2s finished


GridSearchCV(cv=3, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=1000, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=647, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': array([1.00000000e-03, 2.15443469e-02, 4.64158883e-01, 1.00000000e+01]),
                         'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1_macro', verbose=1)

In [20]:
print(gscv.best_params_)

{'C': 0.021544346900318832, 'penalty': 'l2'}


## Just Useful

In [9]:
x_train = train_df['useful'].values
y_train = train_df['stars'].values

In [10]:
x_train.shape, y_train.shape

((10000,), (10000,))

In [11]:
x_train = x_train.reshape(-1, 1)

In [12]:
x_train.shape, y_train.shape

((10000, 1), (10000,))

In [19]:
lr = LogisticRegression(C=0.001)

In [20]:
lr.fit(x_train, y_train)

LogisticRegression(C=0.001, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [21]:
x_valid = valid_df['useful'].values.reshape(-1, 1)
y_valid = valid_df['stars'].values
y_pred = lr.predict(x_valid)
print(classification_report(y_valid, y_pred))
print("\n\n")
print(confusion_matrix(y_valid, y_pred))
print('accuracy', np.mean(y_valid == y_pred))

              precision    recall  f1-score   support

           1       0.26      1.00      0.41       517
           2       0.00      0.00      0.00       278
           3       0.00      0.00      0.00       344
           4       0.00      0.00      0.00       427
           5       0.00      0.00      0.00       434

    accuracy                           0.26      2000
   macro avg       0.05      0.20      0.08      2000
weighted avg       0.07      0.26      0.11      2000




[[517   0   0   0   0]
 [278   0   0   0   0]
 [344   0   0   0   0]
 [427   0   0   0   0]
 [434   0   0   0   0]]
accuracy 0.2585


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
# Grid-search for c
params = dict(C = np.logspace(-3, 1, 4),
              penalty = ['l1', 'l2'])

gscv = GridSearchCV(LogisticRegression(max_iter=1000,
                                       random_state=647),
                    params,
                    scoring='f1_macro',
                    cv=3,
                    verbose=1,
                    n_jobs=-1)

In [23]:
gscv.fit(x_train, y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:    0.6s finished


GridSearchCV(cv=3, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=1000, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=647, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': array([1.00000000e-03, 2.15443469e-02, 4.64158883e-01, 1.00000000e+01]),
                         'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1_macro', verbose=1)

In [24]:
print(gscv.best_params_)

{'C': 0.001, 'penalty': 'l2'}


## All three combined

In [16]:
x_train = np.array([train_df['cool'].values, train_df['funny'].values, train_df['useful'].values])
y_train = train_df['stars'].values

In [17]:
x_train.shape, y_train.shape

((3, 10000), (10000,))

In [18]:
x_train = x_train.reshape(-1, 3)

In [19]:
x_train.shape, y_train.shape

((10000, 3), (10000,))

In [29]:
lr = LogisticRegression(C=0.46415888336127775)

In [30]:
lr.fit(x_train, y_train)

LogisticRegression(C=0.46415888336127775, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [31]:
x_valid = np.array([valid_df['cool'].values, valid_df['funny'].values, valid_df['useful'].values]).reshape(-1, 3)
y_valid = valid_df['stars'].values
y_pred = lr.predict(x_valid)
print(classification_report(y_valid, y_pred))
print("\n\n")
print(confusion_matrix(y_valid, y_pred))
print('accuracy', np.mean(y_valid == y_pred))

              precision    recall  f1-score   support

           1       0.26      1.00      0.41       517
           2       0.00      0.00      0.00       278
           3       0.25      0.00      0.01       344
           4       0.50      0.00      0.00       427
           5       0.25      0.00      0.00       434

    accuracy                           0.26      2000
   macro avg       0.25      0.20      0.09      2000
weighted avg       0.27      0.26      0.11      2000




[[515   0   1   1   0]
 [274   0   2   0   2]
 [342   0   1   0   1]
 [426   0   0   1   0]
 [433   0   0   0   1]]
accuracy 0.259


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [32]:
# Grid-search for c
params = dict(C = np.logspace(-3, 1, 4),
              penalty = ['l1', 'l2'])

gscv = GridSearchCV(LogisticRegression(max_iter=1000,
                                       random_state=647),
                    params,
                    scoring='f1_macro',
                    cv=3,
                    verbose=1,
                    n_jobs=-1)

In [33]:
gscv.fit(x_train, y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:    0.7s finished


GridSearchCV(cv=3, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=1000, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=647, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': array([1.00000000e-03, 2.15443469e-02, 4.64158883e-01, 1.00000000e+01]),
                         'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1_macro', verbose=1)

In [34]:
print(gscv.best_params_)

{'C': 0.46415888336127775, 'penalty': 'l2'}
